## Let's review news from Python 3.5

## `*` Iterable unpacking operator
Allows to extract all elements from a list/iterable

See https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-448

In [1]:
l = [1, 2, 3, 4, 5, 6]
print(l)
print(*l)
a, b, c, _, _, _ = l
print(a, b, c)

[1, 2, 3, 4, 5, 6]
1 2 3 4 5 6
1 2 3


## `**` Dictionary unpacking operator
Same as `*`, but for dictionaries.

See https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-448

In [2]:
d1 = {"key1": "value1", "key2": "value2"}
print(d1)
print(*d1)
# print(**d1) : error, print cannot take so many arguments

d2 = {"key3": "value3", "key3": "value3"}

d11 = {**d1}
print(d11)

d12 = {**d1, **d2}
print(d12)

# print(d1 + d2): + does not know how to add dictionaries

def hello(*args, **kwargs):
    # here, * and ** does reverse-unpackaging!
    print(args) # args: non-named arguments
    print(kwargs) # kwargs: KeyWord arguments
    internal = {"key": "value"}
    new = {**kwargs, **internal}
    print(new)
    
hello(1, 2, 3, x=1, y=3)

{'key1': 'value1', 'key2': 'value2'}
key1 key2
{'key1': 'value1', 'key2': 'value2'}
{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
(1, 2, 3)
{'x': 1, 'y': 3}
{'x': 1, 'y': 3, 'key': 'value'}


## async/await keywords

Asyncio is only suitable for IOs.

`async` allows to define a function that is `await`able.

`await` allows to wait for an `async` function to return.
It allows the event_loop to pause this task, and resume it when a specific event is received.

If a function bogs the CPU, the event_loop running on the same thread will also be stopped to run by the OS.
This is why asyncio should only be used for IOs, and not CPU intensive tasks.

Warning: some libraries might look like only doing IO, but might be actively blocking (ex: python's `requsets`).
Consequence: libraries need to be fully designed with async in mind to work properly.

Following example modified from https://docs.python.org/3/whatsnew/3.5.html#pep-492-coroutines-with-async-and-await-syntax.

In [3]:
import asyncio
import random


async def http_get(domain):
    reader, writer = await asyncio.open_connection(domain, 80) # await = task is paused, will be waked up by an event
    # Here, event was received, and function has been resumed
    # at this point, a connection has been made, nothing has been sent
    
    # We now send a request
    writer.write(b'\r\n'.join([
        b'GET / HTTP/1.1',
        b'Host: %b' % domain.encode('latin-1'),
        b'Connection: close',
        b'', b''
    ]))
    
    # the response is not there yet.
    # We read the response with a reader, when it comes (streamed).
    # Paused while it is in transit
    async for line in reader:
        await asyncio.sleep(random.randint(3, 9) / 100.0) # increase the delay between bits of the stream
        print(f"{domain} >>>", line)

    writer.close()

loop = asyncio.get_event_loop()
# jupyter already is in an asyncio context (Tornado), so cannot create a new event_loop inside
# See https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook
t1 = loop.create_task(http_get("amadeus.com"))
t2 = loop.create_task(http_get("google.co.jp"))
t3 = loop.create_task(http_get("google.de"))
print("after") # probably printed before, as the request is not there yet from the network

# Explicitly wait for all tasks to but finished before running the next cell
await asyncio.gather(t1, t2, t3)

after
google.co.jp >>> b'HTTP/1.1 301 Moved Permanently\r\n'
amadeus.com >>> b'HTTP/1.1 301 Moved Permanently\r\n'
amadeus.com >>> b'Location: https://amadeus.com/\r\n'
google.de >>> b'HTTP/1.1 301 Moved Permanently\r\n'
google.co.jp >>> b'Location: http://www.google.co.jp/\r\n'
amadeus.com >>> b'Content-Length: 0\r\n'
google.de >>> b'Location: http://www.google.de/\r\n'
google.co.jp >>> b'Content-Type: text/html; charset=UTF-8\r\n'
amadeus.com >>> b'Connection: close\r\n'
google.de >>> b'Content-Type: text/html; charset=UTF-8\r\n'
google.co.jp >>> b'Date: Sat, 24 Oct 2020 06:26:32 GMT\r\n'
amadeus.com >>> b'\r\n'
google.de >>> b'Date: Sat, 24 Oct 2020 06:26:32 GMT\r\n'
google.co.jp >>> b'Expires: Mon, 23 Nov 2020 06:26:32 GMT\r\n'
google.de >>> b'Expires: Mon, 23 Nov 2020 06:26:32 GMT\r\n'
google.co.jp >>> b'Cache-Control: public, max-age=2592000\r\n'
google.de >>> b'Cache-Control: public, max-age=2592000\r\n'
google.co.jp >>> b'Server: gws\r\n'
google.de >>> b'Server: gws\r\n'
google

[None, None, None]

## Let's review news from Python 3.6

### String literales (aka f-string)

See https://www.python.org/dev/peps/pep-0498

In [4]:
a = 3
print(f"Hello number {a}")

Hello number 3


### underscore to separate big numbers
See https://www.python.org/dev/peps/pep-0515

In [5]:
print(1_000)

1000


### Async/await can be used with generators (yield): async generators
First, review of a normal generator (here calculating the fibonacci numbers)

In [6]:
import time

def fibonacci(limit: int):
    f0 = 0
    f1 = 1
    fn_1 = f1
    fn_2 = f0
    fn = f0
    while fn < limit:
        yield fn
        fn = fn_1 + fn_2
        fn_2 = fn_1
        fn_1 = fn
        time.sleep(random.randint(3, 9) / 10.0) # Simulate blocking IO
        
        
        
sequence = fibonacci(100)
        
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())

print("with a loop")
for f in fibonacci(100):
    print(f)

0
1
2
3
5
with a loop
0
1
2
3
5
8
13
21
34
55
89


Now, same, but asynchronously.

Not that this also allows `async for` loops using those generators.

See https://www.python.org/dev/peps/pep-0525.

In [7]:
async def fibonacci(limit: int):
    f0 = 0
    f1 = 1
    fn_1 = f1
    fn_2 = f0
    fn = f0
    while fn < limit:
        yield fn
        fn = fn_1 + fn_2
        fn_2 = fn_1
        fn_1 = fn
        await asyncio.sleep(random.randint(3, 9) / 10.0) # Simulate async IO

        
async def my_task():
    sequence = fibonacci(100)
        
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())

    print("with a loop")
    async for f in fibonacci(100):
        print(f)
        
    # also works in list comprehension
    f_numbers = [n async for n in fibonacci(100)]
    print(f_numbers)
        
loop = asyncio.get_event_loop()
t1 = loop.create_task(my_task())
await asyncio.gather(t1)

0
1
2
3
5
with a loop
0
1
2
3
5
8
13
21
34
55
89
[0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


[None]

### Know all subclasses that exist in the code
`__init_subclass__` is called when a subclass is defined (not need to instanciate it).
See https://www.python.org/dev/peps/pep-0487.

In [8]:
class PluginBase:
    subclasses = []

    def __init_subclass__(cls, **kwargs): # called when the subclass is defined, not when instanciated
        super().__init_subclass__(**kwargs)
        cls.subclasses.append(cls)

class Plugin1(PluginBase):
    pass

class Plugin2(PluginBase):
    pass

# Try to un-comment me!
# class Plugin2(PluginBase):
#    pass

print(Plugin1.subclasses) # No need to create an instance to know about all known plugins to Python

[<class '__main__.Plugin1'>, <class '__main__.Plugin2'>]


This allows to put constraints on inheritating classes. For example a "closed class":

In [9]:
class ClosedPluginBase:
    subclasses = []

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        if cls.__name__ not in ["Plugin1", "Plugin4"]:
            raise RuntimeError("Disallowed to create such an instance")
        cls.subclasses.append(cls)

class Plugin1(ClosedPluginBase):
    pass

class Plugin2(ClosedPluginBase): # Exception raised here
    pass

RuntimeError: Disallowed to create such an instance

## Let's review news from Python 3.7